# Introduction
In this tutorial, we'll cover how you can launch LeMa jobs on custom clusters that are not supported out of the box.

Specifically, this tutorial is gears towards individuals who have access to a compute cluster that's not hosted on a common cloud provider (e.g. University compute clusters).

We'll cover the following topics:
1. Prerequisites
1. The LeMa Launcher Hierarchy
1. Creating a CustomClient Class
1. Creating a CustomCluster Class
1. Creating a CustomCloud Class
1. Registering Your CustomCloud
1. Running a Job on Your Cloud

# Prerequisites


## LeMa Installation
First, let's install lema. You can find detailed instructions [here](https://github.com/openlema/lema/blob/main/README.md), but it should be as simple as:

```bash
pip install -e ".[dev,train]"
```

# The LeMa Launcher Hierarchy

### Preface
Before diving into this tutorial, lets discuss the hierarchy of the LeMa Launcher. At this point, it's worth reading through our tutorial on [Running Jobs Remotely](https://github.com/openlema/lema/blob/main/notebooks/LeMa%20-%20Running%20Jobs%20Remotely.ipynb) to better understand the end-to-end flow of the launcher. Already read it? Great!

### Overview
At a high level, the LeMa Launcher is composed of 3 tiers of objects: `Clouds`, `Clusters`, and `Clients`. The Launcher holds an instance of each unique `Cloud`. These `Clouds`, in turn, are responsible for creating compute `Clusters`. And `Clusters` coordinate running jobs. All communication with remote APIs happens via the `Client`.

#### Clouds
A Cloud class must implement the [`BaseCloud`](https://github.com/openlema/lema/blob/main/src/lema/core/types/base_cloud.py) abstract class. The Launcher will only create one instance of each Cloud, so it's important that a single Cloud object is capable of turning up and down multiple clusters.

You can find several implementations of Clouds [here](https://github.com/openlema/lema/tree/main/src/lema/launcher/clouds).

#### Clusters
A Cluster class must implement the [`BaseCluster`](https://github.com/openlema/lema/blob/main/src/lema/core/types/base_cluster.py) abstract class. A cluster represents a single instance of hardware. For a custom clusters (such as having a single super computer), it may be the case that you only need 1 cluster to represent your hardware setup.

You can find several implementations of Clusters [here](https://github.com/openlema/lema/tree/main/src/lema/launcher/clusters).

#### Clients
Clients are a completely optional but highly encouraged class. Clients should encapsulate all logic that calls remote APIs related to your cloud. While this logic could be encapsulated with your Cluster and Cloud classes, having a dedicated class for this purpose greatly simplifies your Cloud and Cluster logic.

You can find several implementations of Clients [here](https://github.com/openlema/lema/tree/main/src/lema/launcher/clients).

# Creating a CustomClient Class
Let's get started by creating a client for our new cloud, `Foobar`. Let's create a simple client that randomly sets the state of the job on submission. It also supports canceling jobs, and turning down clusters:

In [ ]:
import random
from enum import Enum
from typing import List, Optional

from lema.core.types.base_cluster import JobStatus
from lema.core.types.configs import JobConfig


class _JobState(Enum):
    """An enumeration of the possible states of a job."""

    QUEUED = "QUEUED"
    RUNNING = "RUNNING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"
    CANCELED = "CANCELED"


class CustomClient:
    """A client for running jobs locally in a subprocess."""

    def __init__(self):
        """Initializes a new instance of the CustomClient class."""
        self._jobs = []

    def submit_job(self, job: JobConfig) -> JobStatus:
        """Pretends to run the specified job on this cluster."""
        job_id = str(len(self._jobs))
        name = job.name if job.name else job_id
        # Pick a random status
        status = random.choice([state for state in _JobState])
        job_status = JobStatus(
            name=name,
            id=job_id,
            status=status.value,
            cluster="",
            metadata="",
            done=False,
        )
        self._jobs.append(job_status)
        return job_status

    def list_jobs(self) -> List[JobStatus]:
        """Returns a list of job statuses."""
        return self._jobs

    def get_job(self, job_id: str) -> Optional[JobStatus]:
        """Gets the specified job's status.

        Args:
            job_id: The ID of the job to get.

        Returns:
            The job status if found, None otherwise.
        """
        job_list = self.list_jobs()
        for job in job_list:
            if job.id == job_id:
                return job
        return None

    def cancel(self, job_id) -> Optional[JobStatus]:
        """Cancels the specified job.

        Args:
            job_id: The ID of the job to cancel.

        Returns:
            The job status if found, None otherwise.
        """
        if job_id > len(self._jobs):
            return None
        job_status = self._jobs[job_id]
        job_status.status = _JobState.CANCELED.value
        return job_status

    def turndown_cluster(self, cluster_name: str):
        """Turns down the cluster."""
        print(f"Turning down cluster {cluster_name}...")
        pass

# Creating a CustomCluster Class
Now that we have a client that talk's to our API, we can use the Client to build a Cluster!

In [ ]:
from typing import Any, List, Optional

from lema.core.types.base_cluster import BaseCluster


class CustomCluster(BaseCluster):
    """A custom cluster implementation."""

    def __init__(self, name: str, client: CustomClient) -> None:
        """Initializes a new instance of the CustomCluster class."""
        self._name = name
        self._client = client

    def __eq__(self, other: Any) -> bool:
        """Checks if two LocalClusters are equal."""
        if not isinstance(other, CustomCluster):
            return False
        return self.name() == other.name()

    def name(self) -> str:
        """Gets the name of the cluster."""
        return self._name

    def get_job(self, job_id: str) -> Optional[JobStatus]:
        """Gets the jobs on this cluster if it exists, else returns None."""
        for job in self.get_jobs():
            if job.id == job_id:
                return job
        return None

    def get_jobs(self) -> List[JobStatus]:
        """Lists the jobs on this cluster."""
        jobs = self._client.list_jobs()
        for job in jobs:
            job.cluster = self._name
        return jobs

    def stop_job(self, job_id: str) -> JobStatus:
        """Stops the specified job on this cluster."""
        self._client.cancel(job_id)
        job = self.get_job(job_id)
        if job is None:
            raise RuntimeError(f"Job {job_id} not found.")
        return job

    def run_job(self, job: JobConfig) -> JobStatus:
        """Runs the specified job on this cluster.

        Args:
            job: The job to run.

        Returns:
            The job status.
        """
        return self._client.submit_job(job)

    def down(self) -> None:
        """Cancel all jobs and turn down the cluster."""
        for job in self.get_jobs():
            self.stop_job(job.id)
        self._client.turndown_cluster(self._name)